In [45]:
from kmodes.kprototypes import KPrototypes
import numpy as np
import pandas as pd
from time import time

In [67]:
# Se importan los datos para ambas bases de datos
diagnose_df = pd.read_excel("Datos_diagnosticos.xlsx") # Datos de diagnostico de pacientes
process_df = pd.read_excel('Datos_procedimientos.xlsx') # Datos de procedimientos

process_df = process_df.rename({"Case": "CASE"}, axis="columns")

# Se calcula el LOS (en dias) para cada fila de la base de datos de procedimientos
LOS_columns = [(obs[1]["Release"] - obs[1]["Date"]).days for obs in process_df.iterrows()]

# Se inserta una nueva columna a la 'process_df' con los datos de LOS
process_df.insert(4, "LOS", LOS_columns, allow_duplicates=True)

In [68]:
process_df.head(5)

,CASE,Procedure,Date,Release,LOS
0,13872110,0DB64Z3,2018-01-18,2018-01-21,3
1,14035188,0BB64ZZ,2018-01-11,2018-01-14,3
2,14085514,0QP934Z,2018-02-07,2018-02-08,1
3,14111667,0DB64Z3,2018-04-09,2018-04-12,3
4,14111831,0DB64Z3,2018-01-08,2018-01-10,2


In [69]:
diagnose_df.head(5)

,CASE,Seq,PrincSec,Diagnosis
0,13872110,1,P,E6601
1,13872110,1,S,Z6841
2,13872110,1,S,DDDDDD
3,14035188,1,P,J984
4,14035188,1,S,I119


In [70]:
# Se eliminan las columnas de Date y Release
process_df = process_df.drop(['Date', 'Release'], axis=1)

process_df.head(10)

,CASE,Procedure,LOS
0,13872110,0DB64Z3,3
1,14035188,0BB64ZZ,3
2,14085514,0QP934Z,1
3,14111667,0DB64Z3,3
4,14111831,0DB64Z3,2
5,14114821,0H0V0JZ,2
6,14114821,0J0L0ZZ,2
7,14114821,0J0L3ZZ,2
8,14114821,0J0M0ZZ,2
9,14114821,0J0M3ZZ,2


In [71]:
# Se realiza un join de ambas tablas mediante la variable CASE
joined = pd.merge(diagnose_df, process_df, on="CASE")
joined.drop_duplicates(keep=False, inplace=True)

In [72]:
joined.head(5)

,CASE,Seq,PrincSec,Diagnosis,Procedure,LOS
0,13872110,1,P,E6601,0DB64Z3,3
1,13872110,1,S,Z6841,0DB64Z3,3
2,13872110,1,S,DDDDDD,0DB64Z3,3
3,14035188,1,P,J984,0BB64ZZ,3
4,14035188,1,S,I119,0BB64ZZ,3


In [73]:
# Se elimina la columna CASE ya que es un identificador de cada paciente
joined = joined.drop(['CASE'], axis=1)

In [74]:
joined.head(10)

,Seq,PrincSec,Diagnosis,Procedure,LOS
0,1,P,E6601,0DB64Z3,3
1,1,S,Z6841,0DB64Z3,3
2,1,S,DDDDDD,0DB64Z3,3
3,1,P,J984,0BB64ZZ,3
4,1,S,I119,0BB64ZZ,3
5,1,S,E669,0BB64ZZ,3
6,1,S,Z6830,0BB64ZZ,3
7,1,S,DDDDDD,0BB64ZZ,3
8,1,P,S72302E,0QP934Z,1
9,1,S,V299XXD,0QP934Z,1


In [167]:
# Guardamos el diagnostico de acuerdo a su primera letra
diagnose_class = [d[0] for d in joined["Diagnosis"]]

diagnose_LOS = {"Diagnosis": diagnose_class,  
                "PrincSec": list(joined["PrincSec"]),
                "Seq": list(joined['Seq']),
                "Procedure": list(joined['Procedure']),
                "LOS": list(joined["LOS"])}

In [185]:
# Actualizamos el dataframe
joinedClassified = pd.DataFrame(diagnose_LOS) 

joinedClassified.head(10)

,Diagnosis,PrincSec,Seq,Procedure,LOS
0,E,P,1,0DB64Z3,3
1,Z,S,1,0DB64Z3,3
2,D,S,1,0DB64Z3,3
3,J,P,1,0BB64ZZ,3
4,I,S,1,0BB64ZZ,3
5,E,S,1,0BB64ZZ,3
6,Z,S,1,0BB64ZZ,3
7,D,S,1,0BB64ZZ,3
8,S,P,1,0QP934Z,1
9,V,S,1,0QP934Z,1


In [85]:
# Se setean los parámetros de la predicción: 
# n_init: se reinicia el algoritmo 3 veces
# n_clusters: se obtienen 8 clusters
# verbose: se muestra la información de las iteraciones
# max_iter: maximo de iteraciones por algoritmo
kproto = KPrototypes(n_init=3,  n_clusters=8, verbose=2, max_iter=20)

# Se transforma a un ndarray el dataframe
joined_array = joinedClassified.values

In [86]:
t0 = time()
print('Inicializando el modelo...')

# Se aplica el modelo a nuestro arreglo de datos
clusters = kproto.fit_predict(joined_array, categorical=[0,1,3])

t5 = round(time() - t0, 2)
print(f"Se ha demorado {t5} segundos en encontrar los clusters!")

Inicializando el modelo...
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 35700,

In [169]:
print(f"Se demoro {round(t5/60, 1)} minutos en encontrar los clusters!")

# Se imprimen los centroides obtenidos para cada variable (numerica o categorica)
centroids = kproto.cluster_centroids_

print(centroids)

Se demoro 94.8 minutos en encontrar los clusters!
[array([[  2.69225225, 135.24846847],
       [  1.48494109,   2.51488836],
       [  2.4065204 ,  50.76936547],
       [  2.91219744, 206.2586616 ],
       [  2.45990529,  32.42277417],
       [  2.297454  ,  19.18502647],
       [  2.05941312,   9.93804814],
       [  2.69476389,  81.95424517]]), array([['Z', 'S', '30233N1'],
       ['Z', 'S', '4A023N7'],
       ['Z', 'S', '30233N1'],
       ['D', 'S', '3E0G76Z'],
       ['Z', 'S', '30233N1'],
       ['Z', 'S', '30233N1'],
       ['Z', 'S', '30233N1'],
       ['Z', 'S', '30233N1']], dtype='<U7')]


In [186]:
# Agregamos una columna de cluster, que entrega el cluster al que pertenece cada dato
joinedClassified['cluster'] = [c for c in clusters]

In [188]:
# Se genera un diccionario con particiones del dataframe original segun el cluster
clusterDataframes = {i: joinedClassified[joinedClassified['cluster'] == i] for i in range(8)}

In [183]:
example = 10

In [199]:
# Se printean los LOS
losValues_0 = pd.unique(clusterDataframes[0]['LOS'].values)
print(f'Valores de LOS: {losValues_0}')

print(clusterDataframes[0].head(example))

Valores de LOS: [166 163 151 155 148 147 109 150 134 136 133 141 152 146 159 123 140 129
 113 138 143 156 135 115 130 127 131 110 154 137 114 142 132 120 119 139
 124 118 112 122 128 121 111 125 117]
    Diagnosis PrincSec  Seq Procedure  LOS  cluster
101         M        P    1   3E04305  166        0
113         C        S    1   3E04305  166        0
125         G        S    1   3E04305  166        0
137         C        S    1   3E04305  166        0
149         C        S    1   3E04305  166        0
161         I        S    1   3E04305  166        0
173         K        S    1   3E04305  166        0
185         E        S    1   3E04305  166        0
197         F        S    1   3E04305  166        0
209         R        S    1   3E04305  166        0


In [200]:
# Se printean los LOS
losValues_1 = pd.unique(clusterDataframes[1]['LOS'].values)
print(f'Valores de LOS: {losValues_1}')

print(clusterDataframes[1].head(example))

Valores de LOS: [3 1 2 6 4 5 0 7]
  Diagnosis PrincSec  Seq Procedure  LOS  cluster
0         E        P    1   0DB64Z3    3        1
1         Z        S    1   0DB64Z3    3        1
2         D        S    1   0DB64Z3    3        1
3         J        P    1   0BB64ZZ    3        1
4         I        S    1   0BB64ZZ    3        1
5         E        S    1   0BB64ZZ    3        1
6         Z        S    1   0BB64ZZ    3        1
7         D        S    1   0BB64ZZ    3        1
8         S        P    1   0QP934Z    1        1
9         V        S    1   0QP934Z    1        1


In [201]:
# Se printean los LOS
losValues_2 = pd.unique(clusterDataframes[2]['LOS'].values)
print(f'Valores de LOS: {losValues_2}')

print(clusterDataframes[2].head(example))

Valores de LOS: [56 49 50 46 42 63 53 44 43 62 61 64 65 66 58 48 54 59 45 57 60 47 52 51
 55]
    Diagnosis PrincSec  Seq Procedure  LOS  cluster
96          M        P    1   0D1M0Z4   56        2
98          M        P    1   0DTG0ZZ   56        2
102         M        P    1   3E0G76Z   49        2
103         M        P    1   5A1955Z   50        2
105         M        P    1   8E0ZXY6   46        2
108         C        S    1   0D1M0Z4   56        2
110         C        S    1   0DTG0ZZ   56        2
114         C        S    1   3E0G76Z   49        2
115         C        S    1   5A1955Z   50        2
117         C        S    1   8E0ZXY6   46        2


In [202]:
# Se printean los LOS
losValues_3 = pd.unique(clusterDataframes[3]['LOS'].values)
print(f'Valores de LOS: {losValues_3}')

print(clusterDataframes[3].head(example))

Valores de LOS: [186 184 174 180 181 172 214 215 261 262 232 245 231 191 171 224 213 183
 182 193 209 192 194 198 187 219 205 211 212 197 204 188]
    Diagnosis PrincSec  Seq Procedure  LOS  cluster
95          M        P    1   00NW0ZZ  186        3
97          M        P    1   0D968ZZ  184        3
99          M        P    1   0RG20Z1  186        3
100         M        P    1   0RG70Z1  186        3
106         M        P    1   DB021ZZ  174        3
107         C        S    1   00NW0ZZ  186        3
109         C        S    1   0D968ZZ  184        3
111         C        S    1   0RG20Z1  186        3
112         C        S    1   0RG70Z1  186        3
118         C        S    1   DB021ZZ  174        3


In [203]:
# Se printean los LOS
losValues_4 = pd.unique(clusterDataframes[4]['LOS'].values)
print(f'Valores de LOS: {losValues_4}')

print(clusterDataframes[4].head(example))

Valores de LOS: [28 37 30 27 26 39 38 41 31 33 32 36 35 40 29 34]
    Diagnosis PrincSec  Seq Procedure  LOS  cluster
104         M        P    1   5A1955Z   28        4
116         C        S    1   5A1955Z   28        4
128         G        S    1   5A1955Z   28        4
140         C        S    1   5A1955Z   28        4
152         C        S    1   5A1955Z   28        4
164         I        S    1   5A1955Z   28        4
176         K        S    1   5A1955Z   28        4
188         E        S    1   5A1955Z   28        4
200         F        S    1   5A1955Z   28        4
212         R        S    1   5A1955Z   28        4


In [204]:
# Se printean los LOS
losValues_5 = pd.unique(clusterDataframes[5]['LOS'].values)
print(f'Valores de LOS: {losValues_5}')

print(clusterDataframes[5].head(example))

Valores de LOS: [15 24 19 17 18 23 21 16 25 20 22]
     Diagnosis PrincSec  Seq Procedure  LOS  cluster
1338         U        P    1   0DH63UZ   15        5
1339         U        P    1   0DJ08ZZ   15        5
1351         I        P    2   0DH63UZ   15        5
1352         I        P    2   0DJ08ZZ   15        5
1364         I        S    2   0DH63UZ   15        5
1365         I        S    2   0DJ08ZZ   15        5
1377         G        S    2   0DH63UZ   15        5
1378         G        S    2   0DJ08ZZ   15        5
1390         G        S    2   0DH63UZ   15        5
1391         G        S    2   0DJ08ZZ   15        5


In [205]:
# Se printean los LOS
losValues_6 = pd.unique(clusterDataframes[6]['LOS'].values)
print(f'Valores de LOS: {losValues_6}')

print(clusterDataframes[6].head(example))

Valores de LOS: [ 7 14  9 12  8  6 10 13 11]
     Diagnosis PrincSec  Seq Procedure  LOS  cluster
2342         E        P    1   0JB80ZZ    7        6
2343         E        P    1   0JD83ZZ    7        6
2344         D        S    1   0JB80ZZ    7        6
2345         D        S    1   0JD83ZZ    7        6
5101         U        P    1   3E0G36Z   14        6
5104         U        P    1   F07Z9ZZ   14        6
5108         T        P    2   3E0G36Z   14        6
5111         T        P    2   F07Z9ZZ   14        6
5115         W        S    2   3E0G36Z   14        6
5118         W        S    2   F07Z9ZZ   14        6


In [206]:
# Se printean los LOS
losValues_7 = pd.unique(clusterDataframes[7]['LOS'].values)
print(f'Valores de LOS: {losValues_7}')

print(clusterDataframes[7].head(example))

Valores de LOS: [ 81  76 102 101  90  83 107 104  95 105  99  71  98  91  70  97  84  73
  87  78  80  79  89  69  68  92 100  96  82  74  85 108  94  86  72  67
  75  77 103  93  88 106]
     Diagnosis PrincSec  Seq Procedure  LOS  cluster
2234         U        P    1   0DB60ZZ   81        7
2236         U        P    1   0DH60UZ   81        7
2237         U        P    1   0FT40ZZ   81        7
2238         C        P    2   0DB60ZZ   81        7
2240         C        P    2   0DH60UZ   81        7
2241         C        P    2   0FT40ZZ   81        7
2242         I        S    2   0DB60ZZ   81        7
2244         I        S    2   0DH60UZ   81        7
2245         I        S    2   0FT40ZZ   81        7
2246         D        S    2   0DB60ZZ   81        7
